# IBM Data Science
This is the Jupyter Notebook for the Applied Data Science Capstone by Christoph Yew.
In this project I will try to find the best location for building a train station in Kuala Lumpur based on the locations of shopping malls, office towers.

In [1]:
# Install all kinds of necessary libraries
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!pip install folium==0.5.0
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

     |████████████████████████████████| 81kB 10.8MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium
Folium installed
Libraries imported.


In [2]:
#Setup my Foursquare credentials
#CLIENT_ID = 'BRF1SHWY1JDITTS104HAIHQ3W2INU45FZUMIKTPE5XQW0IDV' # your Foursquare ID
#CLIENT_SECRET = 'JIEIZ0QE0OTWRVIRUTDGA4NIZWA2JYZ5DOTZIWUMIFOY11OP' # your Foursquare Secret

CLIENT_ID = 'BXT5F1JDCYKBHPGGE2MEZVH0U23O50Q4BSIOQACWWTWMA4MZ'
CLIENT_SECRET = 'ZLBJ4XLOMEIPNZDV3VDAACYGHU5Z2NUQ0NCFTT2DZH21ROIU'
VERSION = '20200608'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BXT5F1JDCYKBHPGGE2MEZVH0U23O50Q4BSIOQACWWTWMA4MZ
CLIENT_SECRET:ZLBJ4XLOMEIPNZDV3VDAACYGHU5Z2NUQ0NCFTT2DZH21ROIU


In [3]:
#Get the location data of Kuala Lumpur
address = 'Kuala Lumpur'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
print(location)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

Kuala Lumpur, Malaysia
3.1516964 101.6942371


In [4]:
#Scrape the website with the neighborhood data to generate a list of neighborhoods in Kuala Lumpur and display the l
from bs4 import BeautifulSoup
html_doc = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text
soup = BeautifulSoup(html_doc, 'html.parser')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   Category:Suburbs in Kuala Lumpur - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"98751ddf-9110-4e42-91a1-9a00cbb8e41b","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Suburbs_in_Kuala_Lumpur","wgTitle":"Suburbs in Kuala Lumpur","wgCurRevisionId":959820832,"wgRevisionId":959820832,"wgArticleId":9710314,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Geography of Kuala Lumpur","Suburbs by city"],"wgPageContentLanguage":"en","wgPageContentModel

In [5]:
#The neighborhoods are located in class mw-category-generated as a list. Get them out of the html text and put them into a Python list.
neighborhoods = []

for row in soup.find_all("div", class_="mw-category-generated")[0].findAll("li"):
    #neighborhoods.append(row.text)
    if ", Kuala Lumpur" in row.text:
        neighborhoods.append(row.text + ", Malaysia")
    else:
        neighborhoods.append(row.text + ", Kuala Lumpur, Malaysia")

print(neighborhoods)

['Alam Damai, Kuala Lumpur, Malaysia', 'Ampang, Kuala Lumpur, Malaysia', 'Bandar Menjalara, Kuala Lumpur, Malaysia', 'Bandar Sri Permaisuri, Kuala Lumpur, Malaysia', 'Bandar Tasik Selatan, Kuala Lumpur, Malaysia', 'Bandar Tun Razak, Kuala Lumpur, Malaysia', 'Bangsar, Kuala Lumpur, Malaysia', 'Bangsar Park, Kuala Lumpur, Malaysia', 'Bangsar South, Kuala Lumpur, Malaysia', 'Batu 11 Cheras, Kuala Lumpur, Malaysia', 'Batu, Kuala Lumpur, Malaysia', 'Brickfields, Kuala Lumpur, Malaysia', 'Bukit Bintang, Kuala Lumpur, Malaysia', 'Bukit Jalil, Kuala Lumpur, Malaysia', 'Bukit Kiara, Kuala Lumpur, Malaysia', 'Bukit Nanas, Kuala Lumpur, Malaysia', 'Bukit Petaling, Kuala Lumpur, Malaysia', 'Bukit Tunku, Kuala Lumpur, Malaysia', 'Cheras, Kuala Lumpur, Malaysia', 'Chow Kit, Kuala Lumpur, Malaysia', 'Damansara Heights, Kuala Lumpur, Malaysia', 'Damansara Town Centre, Kuala Lumpur, Malaysia', 'Damansara, Kuala Lumpur, Malaysia', 'Dang Wangi, Kuala Lumpur, Malaysia', 'Desa Petaling, Kuala Lumpur, Malay

In [6]:
#Create a dataframe with the list. It generates a dataframe with 71 entries
pd.set_option('display.max_rows',80)
neighborhoods_df = pd.DataFrame(neighborhoods) 
neighborhoods_df = neighborhoods_df.rename(columns={0: "Neighborhood"})
neighborhoods_df.head(71)

,Neighborhood
0,"Alam Damai, Kuala Lumpur, Malaysia"
1,"Ampang, Kuala Lumpur, Malaysia"
2,"Bandar Menjalara, Kuala Lumpur, Malaysia"
3,"Bandar Sri Permaisuri, Kuala Lumpur, Malaysia"
4,"Bandar Tasik Selatan, Kuala Lumpur, Malaysia"
5,"Bandar Tun Razak, Kuala Lumpur, Malaysia"
6,"Bangsar, Kuala Lumpur, Malaysia"
7,"Bangsar Park, Kuala Lumpur, Malaysia"
8,"Bangsar South, Kuala Lumpur, Malaysia"
9,"Batu 11 Cheras, Kuala Lumpur, Malaysia"


In [8]:
#Get the location data of the neighborhoods and input into dataframe
latitude_list = []
longitude_list = []
for index in neighborhoods:
    location = geolocator.geocode(index)

    try:
        latitude_list.append(location.latitude)
        longitude_list.append(location.longitude)
    except:
        try:
            update_index = index.replace(', Kuala Lumpur, Malaysia','')
            location = geolocator.geocode(update_index)
            latitude_list.append(location.latitude)
            longitude_list.append(location.longitude)
        except:
            latitude_list.append("NaN")
            longitude_list.append("NaN")
        

In [9]:
#Put the neighborhoods and location data into one dataframe
latitude_df = pd.DataFrame(latitude_list) 
longitude_df = pd.DataFrame(longitude_list) 

neighborhoods_df = pd.concat([neighborhoods_df, latitude_df], axis=1, ignore_index=True)
neighborhoods_df = pd.concat([neighborhoods_df, longitude_df], axis=1, ignore_index=True)
neighborhoods_df = neighborhoods_df.rename(columns={0: "Neighborhood", 1:"Latitude", 2:"Longitude"})
neighborhoods_df

,Neighborhood,Latitude,Longitude
0,"Alam Damai, Kuala Lumpur, Malaysia",3.06357,101.739
1,"Ampang, Kuala Lumpur, Malaysia",3.15213,101.732
2,"Bandar Menjalara, Kuala Lumpur, Malaysia",3.19414,101.634
3,"Bandar Sri Permaisuri, Kuala Lumpur, Malaysia",3.10021,101.718
4,"Bandar Tasik Selatan, Kuala Lumpur, Malaysia",3.0761,101.711
5,"Bandar Tun Razak, Kuala Lumpur, Malaysia",3.08969,101.712
6,"Bangsar, Kuala Lumpur, Malaysia",3.13083,101.669
7,"Bangsar Park, Kuala Lumpur, Malaysia",3.13083,101.669
8,"Bangsar South, Kuala Lumpur, Malaysia",3.11297,101.667
9,"Batu 11 Cheras, Kuala Lumpur, Malaysia",3.04133,101.773


In [10]:
#It seems Taman Taynton View cannot be found using Geolocator. I am therefore updating manually.
#3.0879, 101.7373
neighborhoods_df.iloc[65, neighborhoods_df.columns.get_loc('Latitude')] = 3.0879
neighborhoods_df.iloc[65, neighborhoods_df.columns.get_loc('Longitude')] = 101.7373

In [11]:
#Check the dataframe. Looks good.
neighborhoods_df

,Neighborhood,Latitude,Longitude
0,"Alam Damai, Kuala Lumpur, Malaysia",3.06357,101.739
1,"Ampang, Kuala Lumpur, Malaysia",3.15213,101.732
2,"Bandar Menjalara, Kuala Lumpur, Malaysia",3.19414,101.634
3,"Bandar Sri Permaisuri, Kuala Lumpur, Malaysia",3.10021,101.718
4,"Bandar Tasik Selatan, Kuala Lumpur, Malaysia",3.0761,101.711
5,"Bandar Tun Razak, Kuala Lumpur, Malaysia",3.08969,101.712
6,"Bangsar, Kuala Lumpur, Malaysia",3.13083,101.669
7,"Bangsar Park, Kuala Lumpur, Malaysia",3.13083,101.669
8,"Bangsar South, Kuala Lumpur, Malaysia",3.11297,101.667
9,"Batu 11 Cheras, Kuala Lumpur, Malaysia",3.04133,101.773


# Create map of Kuala Lumpur

In [54]:
# Create a map of Kuala Lumpur and Zoom-in
map_kl = folium.Map(location=[latitude, longitude], zoom_start=12)

# Add neighborhoods to map
for lat, lng, neighborhood in zip(neighborhoods_df['Latitude'], neighborhoods_df['Longitude'], neighborhoods_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8).add_to(map_kl)  
    
map_kl

# Locate Offices and Shopping Spaces
Here I will locate office and shopping spaces and put their names and location data into one dataframe.

In [13]:
#Install normalize to turn json data into a dataframe 
!pip install --user pandas==1.0.5

import pandas as pd
from pandas.io.json import json_normalize

     |████████████████████████████████| 10.1MB 10.1MB/s eta 0:00:01
ERROR: autoai-libs 1.10.5 has requirement numpy>=1.16.4, but you'll have numpy 1.15.4 which is incompatible.


In [14]:
#Find shopping centres in the neighborhoods
radius = 2000
LIMIT = 200
search_query = 'Shopping'

for lat, long, neighborhood in zip(neighborhoods_df['Latitude'], neighborhoods_df['Longitude'], neighborhoods_df['Neighborhood']):
    
    #create the API request URL
    #url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        #CLIENT_ID, CLIENT_SECRET, VERSION, lat, long, radius,  LIMIT)
    
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    
    # make the GET request
results = requests.get(url).json()

In [15]:
print(results)

{'meta': {'code': 200, 'requestId': '5f072075889e093619651668'}, 'response': {'venues': [{'id': '4b058805f964a520f2ac22e3', 'name': 'Lot 10 Shopping Centre', 'location': {'address': '50, Jalan Sultan Ismail', 'lat': 3.1466184082094646, 'lng': 101.71179285847548, 'labeledLatLngs': [{'label': 'display', 'lat': 3.1466184082094646, 'lng': 101.71179285847548}], 'distance': 2031, 'postalCode': '50250', 'cc': 'MY', 'city': 'Kuala Lumpur', 'state': 'Kuala Lumpur', 'country': 'Malaysia', 'formattedAddress': ['50, Jalan Sultan Ismail', '50250 Kuala Lumpur, Kuala Lumpur', 'Malaysia']}, 'categories': [{'id': '4bf58dd8d48988d1fd941735', 'name': 'Shopping Mall', 'pluralName': 'Shopping Malls', 'shortName': 'Mall', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_', 'suffix': '.png'}, 'primary': True}], 'referralId': 'v-1594302524', 'hasPerk': False}, {'id': '4b058805f964a5202ead22e3', 'name': 'Campbell Shopping Complex', 'location': {'address': '98, Jalan Dang Wangi', 'lat': 3.1

In [16]:
#Turn the shopping centre data into a dataframe
from pandas.io.json import json_normalize
result = json_normalize(results['response']['venues'])
result

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4b058805f964a520f2ac22e3,"50, Jalan Sultan Ismail",MY,Kuala Lumpur,Malaysia,NaN,2031,"[50, Jalan Sultan Ismail, 50250 Kuala Lumpur, ...","[{'label': 'display', 'lat': 3.146618408209464...",3.146618,101.711793,NaN,50250,Kuala Lumpur,Lot 10 Shopping Centre,v-1594302524
1,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4b058805f964a5202ead22e3,"98, Jalan Dang Wangi",MY,Kuala Lumpur,Malaysia,NaN,626,"[98, Jalan Dang Wangi, 50100 Kuala Lumpur, Kua...","[{'label': 'display', 'lat': 3.156821579910517...",3.156822,101.696562,NaN,50100,Kuala Lumpur,Campbell Shopping Complex,v-1594302524
2,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4b0b7f81f964a520e93123e3,Kuala Lumpur City Centre,MY,Kuala Lumpur,Malaysia,Jalan Ampang,2123,"[Kuala Lumpur City Centre (Jalan Ampang), 5008...",NaN,3.157402,101.712464,NaN,50088,Kuala Lumpur,Suria KLCC,v-1594302524
3,[],False,54f296f3498edb39ef5ae409,NaN,MY,NaN,Malaysia,NaN,415,[Malaysia],"[{'label': 'display', 'lat': 3.153948138947948...",3.153948,101.697214,NaN,NaN,NaN,Haniffa One Stop Shopping Center,v-1594302524
4,"[{'id': '4bf58dd8d48988d1bc941735', 'name': 'C...",False,5083c5d5e4b0d2d66469f0ce,Bangsar,MY,NaN,Malaysia,NaN,439,"[Bangsar, 59200, Malaysia]","[{'label': 'display', 'lat': 3.15384816214098,...",3.153848,101.697553,NaN,59200,NaN,Just heaven bangsar shopping compleks,v-1594302524
5,"[{'id': '4bf58dd8d48988d175941735', 'name': 'G...",False,5321d2cc498e760527098246,190,MY,Kuala Lumpur,Malaysia,Jalan Tuanku Abdul Rahman,496,"[190 (Jalan Tuanku Abdul Rahman), 50100 Kuala ...","[{'label': 'display', 'lat': 3.156120952842391...",3.156121,101.694829,NaN,50100,Selangor,Peak Physique Health & Fitness (8th Floor KLSO...,v-1594302524
6,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",False,4e25200e18388c58be293846,Semua House Shopping Complex,MY,NaN,Malaysia,NaN,458,"[Semua House Shopping Complex, Malaysia]","[{'label': 'display', 'lat': 3.154602300845475...",3.154602,101.697167,NaN,NaN,NaN,Sagar Cafe @ Semua House Shopping Complex,v-1594302524
7,"[{'id': '4bf58dd8d48988d1d6941735', 'name': 'S...",False,4e18445e45dd120c120c9e52,"190, Jalan Tuanku Abdul Rahman",MY,Kuala Lumpur,Malaysia,Off Jalan Dang Wangi,551,"[190, Jalan Tuanku Abdul Rahman (Off Jalan Dan...","[{'label': 'display', 'lat': 3.156288, 'lng': ...",3.156288,101.696089,NaN,NaN,Federal Territory of Kuala Lum,Autobots Hideout @ Sogo Shopping Complex,v-1594302524
8,"[{'id': '4bf58dd8d48988d1f6941735', 'name': 'D...",False,4bc69f3204e8b713c35d362d,"190, Jalan Tuanku Abdul Rahman",MY,Kuala Lumpur,Malaysia,Jalan TAR,507,"[190, Jalan Tuanku Abdul Rahman (Jalan TAR), 5...",NaN,3.156096,101.695441,NaN,50100,Kuala Lumpur,KL SOGO,v-1594302524
9,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4b058806f964a5203ead22e3,"Jalan Cheng Lock,",MY,Kuala Lumpur,Malaysia,NaN,808,"[Jalan Cheng Lock,, 50000 Kuala Lumpur, Kuala ...","[{'label': 'display', 'lat': 3.145528577452310...",3.145529,101.698081,NaN,50000,Kuala Lumpur,S & M Shopping Arcade,v-1594302524


In [18]:
#Remove columns which we don't need for our project
result_df = result.drop(['categories', 'hasPerk','id','location.address','location.cc','location.city','location.country','location.crossStreet','location.distance','location.formattedAddress','location.labeledLatLngs','location.postalCode','location.state','referralId'], axis=1)

result_df = result_df.rename(columns={'location.lat': "Latitude"})
result_df = result_df.rename(columns={'location.lng': "Longitude"})
result_df = result_df.rename(columns={'name': "Name"})
result_df = result_df[['Name', 'Latitude', 'Longitude']]
result_df

,Name,Latitude,Longitude
0,Lot 10 Shopping Centre,3.146618,101.711793
1,Campbell Shopping Complex,3.156822,101.696562
2,Suria KLCC,3.157402,101.712464
3,Haniffa One Stop Shopping Center,3.153948,101.697214
4,Just heaven bangsar shopping compleks,3.153848,101.697553
5,Peak Physique Health & Fitness (8th Floor KLSO...,3.156121,101.694829
6,Sagar Cafe @ Semua House Shopping Complex,3.154602,101.697167
7,Autobots Hideout @ Sogo Shopping Complex,3.156288,101.696089
8,KL SOGO,3.156096,101.695441
9,S & M Shopping Arcade,3.145529,101.698081


In [19]:
#Find office spaces as well
radius = 2000
LIMIT = 200
search_query = 'Office'

for lat, long, neighborhood in zip(neighborhoods_df['Latitude'], neighborhoods_df['Longitude'], neighborhoods_df['Neighborhood']):
    
    url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
    
    # make the GET request
results_office = requests.get(url).json()

In [20]:
print(results_office)

{'meta': {'code': 200, 'requestId': '5f0720da6da43a2157e041f7'}, 'response': {'venues': [{'id': '53676053498e76b3a9cefcde', 'name': 'General Post Office (Pejabat Pos Besar)', 'location': {'address': 'Jalan Sultan Hishamuddin', 'lat': 3.143901319048451, 'lng': 101.69396996498106, 'labeledLatLngs': [{'label': 'display', 'lat': 3.143901319048451, 'lng': 101.69396996498106}], 'distance': 868, 'postalCode': '50670', 'cc': 'MY', 'city': 'Kuala Lumpur', 'state': 'Kuala Lumpur', 'country': 'Malaysia', 'formattedAddress': ['Jalan Sultan Hishamuddin', '50670 Kuala Lumpur, Kuala Lumpur', 'Malaysia']}, 'categories': [{'id': '4bf58dd8d48988d172941735', 'name': 'Post Office', 'pluralName': 'Post Offices', 'shortName': 'Post Office', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/postoffice_', 'suffix': '.png'}, 'primary': True}], 'referralId': 'v-1594302771', 'hasPerk': False}, {'id': '4e150a6c18a8addae8c6aed9', 'name': 'Kwsp Equity Research (Office Wan Cool Art)', 'location': {

In [21]:
result_office = json_normalize(results_office['response']['venues'])
result_office

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d172941735', 'name': 'P...",False,53676053498e76b3a9cefcde,Jalan Sultan Hishamuddin,MY,Kuala Lumpur,Malaysia,NaN,868,"[Jalan Sultan Hishamuddin, 50670 Kuala Lumpur,...","[{'label': 'display', 'lat': 3.143901319048451...",3.143901,101.693970,NaN,50670,Kuala Lumpur,General Post Office (Pejabat Pos Besar),v-1594302771
1,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,4e150a6c18a8addae8c6aed9,24 Floor,MY,Wilayah Persekutuan,Malaysia,NaN,261,"[24 Floor, 50530 Wilayah Persekutuan, Kuala Lu...","[{'label': 'display', 'lat': 3.154010625299993...",3.154011,101.694660,NaN,50530,Kuala Lumpur,Kwsp Equity Research (Office Wan Cool Art),v-1594302771
2,"[{'id': '4f04b1572fb6e1c99f3db0bf', 'name': 'S...",False,4f6afd4de4b07984c1328da0,Bank Negara Malaysia,MY,Kuala Lumpur,Malaysia,NaN,400,"[Bank Negara Malaysia, 50929 Kuala Lumpur, Kua...","[{'label': 'display', 'lat': 3.154497973966482...",3.154498,101.691972,NaN,50929,Kuala Lumpur,Maintenance Office,v-1594302771
3,"[{'id': '4bf58dd8d48988d197941735', 'name': 'C...",False,4f17cbdde4b0e5837b7e54d7,33-35,MY,Kuala Lumpur,Malaysia,Jalan Hang Lekiu,584,"[33-35 (Jalan Hang Lekiu), Kuala Lumpur, WPKL,...","[{'label': 'display', 'lat': 3.148254352205994...",3.148254,101.698204,NaN,NaN,WPKL,SEGi University College Office,v-1594302771
4,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4ec5ca160e6158bafc957cfd,"No.8, Lorong Ceylon,",MY,Kuala Lumpur,Malaysia,Lorong Ceylon Off Jalan Raja Chulan,1521,"[No.8, Lorong Ceylon, (Lorong Ceylon Off Jalan...","[{'label': 'display', 'lat': 3.148694674278082...",3.148695,101.707596,NaN,50250,Kuala Lumpur,Seri Bukit Ceylon Office & Residences,v-1594302771
5,"[{'id': '4bf58dd8d48988d174941735', 'name': 'C...",False,4c0ca4cab4c6d13ace750c30,"Block B, Capital Square Center",MY,Kuala Lumpur,Malaysia,Jalan Munshi Abdullah,619,"[Block B, Capital Square Center (Jalan Munshi ...","[{'label': 'display', 'lat': 3.154056898282254...",3.154057,101.699281,NaN,50100,Kuala Lumpur,Signature Office,v-1594302771
6,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,4d928431a79d530c7c64ff72,No. 1 Jalan Nagasari (Off Jalan Raja Chulan),MY,Kuala Lumpur,Malaysia,NaN,1612,"[No. 1 Jalan Nagasari (Off Jalan Raja Chulan),...","[{'label': 'display', 'lat': 3.148792537798102...",3.148793,101.708446,NaN,502000,Kuala Lumpur,The Lion Group - One Residency (Office Tower),v-1594302771
7,"[{'id': '4bf58dd8d48988d130941735', 'name': 'B...",False,4fec158ee4b0bda56bda7aca,NaN,MY,NaN,Malaysia,NaN,1392,[Malaysia],"[{'label': 'display', 'lat': 3.145897894867363...",3.145898,101.705337,NaN,NaN,NaN,Office Tung Shin Hospital 3ATh,v-1594302771
8,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4bdf9bd7e75c0f47eae5cb03,"Level 8, Jln Sultan Hishamuddin",MY,Kuala Lumpur,Malaysia,NaN,930,"[Level 8, Jln Sultan Hishamuddin, 50050 Kuala ...","[{'label': 'display', 'lat': 3.143340803241289...",3.143341,101.694396,NaN,50050,Kuala Lumpur,GTS Office,v-1594302771
9,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O...",False,4f9df831e4b05baf7992cb50,NaN,MY,NaN,Malaysia,NaN,92,[Malaysia],"[{'label': 'display', 'lat': 3.151109838514364...",3.151110,101.693644,NaN,NaN,NaN,office ir hj buden,v-1594302771


In [22]:
result_office_df = result_office.drop(['categories', 'hasPerk','id','location.address','location.cc','location.city','location.country','location.crossStreet','location.distance','location.formattedAddress','location.labeledLatLngs','location.postalCode','location.state','referralId'], axis=1)

result_office_df = result_office_df.rename(columns={'location.lat': "Latitude"})
result_office_df = result_office_df.rename(columns={'location.lng': "Longitude"})
result_office_df = result_office_df.rename(columns={'name': "Name"})
result_office_df = result_office_df[['Name', 'Latitude', 'Longitude']]
result_office_df

,Name,Latitude,Longitude
0,General Post Office (Pejabat Pos Besar),3.143901,101.693970
1,Kwsp Equity Research (Office Wan Cool Art),3.154011,101.694660
2,Maintenance Office,3.154498,101.691972
3,SEGi University College Office,3.148254,101.698204
4,Seri Bukit Ceylon Office & Residences,3.148695,101.707596
5,Signature Office,3.154057,101.699281
6,The Lion Group - One Residency (Office Tower),3.148793,101.708446
7,Office Tung Shin Hospital 3ATh,3.145898,101.705337
8,GTS Office,3.143341,101.694396
9,office ir hj buden,3.151110,101.693644


In [23]:
#Concatenate the two dataframes of shops and offices into one
shoff_df = pd.concat([result_df, result_office_df])
print(shoff_df)

                                                 Name  Latitude   Longitude
0                              Lot 10 Shopping Centre  3.146618  101.711793
1                           Campbell Shopping Complex  3.156822  101.696562
2                                          Suria KLCC  3.157402  101.712464
3                    Haniffa One Stop Shopping Center  3.153948  101.697214
4               Just heaven bangsar shopping compleks  3.153848  101.697553
5   Peak Physique Health & Fitness (8th Floor KLSO...  3.156121  101.694829
6           Sagar Cafe @ Semua House Shopping Complex  3.154602  101.697167
7            Autobots Hideout @ Sogo Shopping Complex  3.156288  101.696089
8                                             KL SOGO  3.156096  101.695441
9                               S & M Shopping Arcade  3.145529  101.698081
10                      ZAWARA Busana Shopping Centre  3.157832  101.695881
11                A Cut Above Bangsar Shopping Centre  3.155525  101.700295
12          

# Visualize the venues on a map

In [32]:
# Create a map of Kuala Lumpur and Zoom-in
map_kl_shoff = folium.Map(location=[latitude, longitude], zoom_start=14)

# Add neighborhoods to map
for lat, lng, name in zip(shoff_df['Latitude'], shoff_df['Longitude'], shoff_df['Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8).add_to(map_kl_shoff)  
    
map_kl_shoff

# Where is the best location?
We can already see that most shopping centres and office spaces are located in the city centre. 
Now lets find the most central location among these spaces.

In [45]:
shoff_df.mean()

Latitude       3.150112
Longitude    101.700595
dtype: float64

In [57]:
train_station_lat = shoff_df.mean().Latitude
train_station_lng = shoff_df.mean().Longitude
train_station_list = [["Best Train Station", train_station_lat, train_station_lng]]

train_station_df = pd.DataFrame(train_station_list) 
train_station_df = train_station_df.rename(columns={0: "Name"})
train_station_df = train_station_df.rename(columns={1: "Latitude"})
train_station_df = train_station_df.rename(columns={2: "Longitude"})

train_station_df

,Name,Latitude,Longitude
0,Best Train Station,3.150112,101.700595


In [65]:
# Create a map of Kuala Lumpur and Zoom-in
map_kl_shoff = folium.Map(location=[latitude, longitude], zoom_start=14)

# Add neighborhoods to map
for lat, lng, name in zip(shoff_df['Latitude'], shoff_df['Longitude'], shoff_df['Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8).add_to(map_kl_shoff)  

for lat, lng, name in zip(train_station_df['Latitude'], train_station_df['Longitude'], train_station_df['Name']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='#00ff26',
        fill_opacity=1.0).add_to(map_kl_shoff)  
    
map_kl_shoff

There you go. We found the best car park location in Kuala Lumpur, Malaysia.